In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
sequences, labels = [], []
# loop each action
for action in actions:
    # Loop through the each action sequence(videos)
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # this array will hold all the frame in one video
        window = []
        # Loop though the each frame 30(Frame)
        for frame_num in range(sequence_length):
            # using np.load function load the saved np array from folder (path,which action,which video number,which frame number)
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # add loaded framee to window
            window.append(res)
        #    
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
# shape should be 90 videos,30 frame each,1353 key points in a frame
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
from tensorflow.keras.models import Sequential
# Make us able to action detection
from tensorflow.keras.layers import LSTM, Dense
# Make us able to monitor our modle during taining
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# load Sequential api
model = Sequential()
# add 3 layers of lstm model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

In [ ]:
model.save('action.h5')